In [75]:
# Configuration -
num_ex = 5000000
famTHR = num_ex*200/500000
num_epochs=1
#pandas read - ,nrows=num_ex really speeds it up !

In [76]:
import re
import os
import string
import pickle
import tempfile
import numpy  as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plot

tf.logging.set_verbosity(tf.logging.ERROR)

def save_obj(obj, name ,overwrite=1):
	filename='data/'+ name + '.pkl';
	if(overwrite==1 and os.path.exists(name)):
		return [];
	with open(filename, 'wb') as f:
		pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
	filename='data/'+ name + '.pkl';
	# if(not os.path.exists(name)):
	# 	return [];
	with open('data/' + name + '.pkl', 'rb') as f:
		return pickle.load(f)

fam='Cross-reference (Pfam)'
main =pd.read_table("./data/uniprot-all.tab.gz", sep='\t',nrows=num_ex)
main[fam]= main[fam].apply(lambda x:str(x).split(';')[0])
alphafam = list(main[fam].unique())
alphabet =list(string.ascii_uppercase)

In [79]:
seq_records = main[['Sequence',fam]]
seq_records.columns=['seq','fam']
# seq_records.dropna() would do the same?!
# col = (seq_records['fam'].isnull()==False) & (seq_records['seq'].isnull()==False)
# seq_records = seq_records[col]
seq_records = seq_records.dropna(how='any')
sample = seq_records#.sample(num_ex)

In [80]:
#All the heavy jobs here -
seq_records = sample.copy()
seq_records['seq']= seq_records['seq'].apply(lambda seq : list(map(lambda x : 1+alphabet.index(x),list(seq))))
seq_records['fam']= seq_records['fam'].apply(lambda x : 1+alphafam.index(x))
#added 1+ for skipping 0 index
counts = seq_records.groupby(['fam']).size().reset_index(name='counts')

In [ ]:
#10801 families.
# counts = counts[ (counts['counts']>=famTHR) & (counts['counts'] < 3400) ]
counts = counts[counts['counts'] > 3300 ]
#608 families having count between 200 and 3400
filtered_fams = list(counts['fam'])
#323162 examples total satisfy it.
seq_records = seq_records[ seq_records['fam'].isin(filtered_fams)]
seq_records

In [146]:
"""
x = pd.DataFrame([[1],[2],[1],[3]])
x = x[(x==1) | (x==2)]
x.dropna()
"""
"""
SequenceData can be stored into TFRecords that is tf's own storage. 
It is easier to handle and reusable
& it follows the protocol buffer format already
"""
# efficient storage of the sequences
# per time step variable no of features.
"""
Padding = 
    static padding = using FIFOQueue
    dynamic padding = using tf.train.next_batch(..,dynamic_pad=True)
    Bucketting = tf.contrib.training.bucket_by_sequence_length(..,dynamic_pad=True)
    
Goal : Handle sequences of unknown lengths
tf.while_loop = dynamic loops and supports backprop grad descent
tf.while_loop(cond_fn,body_fn,loop_vars,)

But now to be able to process Slices of Tensors, use tf.TensorArray()
num_rows = matrix.shape[0]
ta = tf.TensorArray(tf.float32,size=num_rows)
loadedMatrix = ta.unstack(matrix)
read_row = loadedMatrix.read(row_idx)
loadedMatrix = loadedMatrix.write(row_idx,fn(read_row))
matrix = loadedMatrix.stack()

"""
    

'\nPadding = \n    static padding = using FIFOQueue\n    dynamic padding = using tf.train.next_batch(..,dynamic_pad=True)\n    Bucketting = tf.contrib.training.bucket_by_sequence_length(..,dynamic_pad=True)\n    \nGoal : Handle sequences of unknown lengths\ntf.while_loop = dynamic loops and supports backprop grad descent\ntf.while_loop(cond_fn,body_fn,loop_vars,)\n\nBut now to be able to process Slices of Tensors, use tf.TensorArray()\nnum_rows = matrix.shape[0]\nta = tf.TensorArray(tf.float32,size=num_rows)\nloadedMatrix = ta.unstack(matrix)\nread_row = loadedMatrix.read(row_idx)\nloadedMatrix = loadedMatrix.write(row_idx,fn(read_row))\nmatrix = loadedMatrix.stack()\n\n'

In [12]:

# Define how to parse the example - In the way you stored thru the recordWriter
context_features = {
    "length": tf.FixedLenFeature([], dtype=tf.int64)
}
sequence_features = {
    "tokens": tf.FixedLenSequenceFeature([], dtype=tf.int64),
    "labels": tf.FixedLenSequenceFeature([], dtype=tf.int64)
}
def make_example(sequence, labels):
    # The object we return
    ex = tf.train.SequenceExample()
    # A non-sequential feature of our example
    sequence_length = len(sequence)
    ex.context.feature["length"].int64_list.value.append(sequence_length)
# Since only one label will be there now, take the output of last hidden layer
    
    # Feature lists for the two sequential features of our example
    fl_tokens = ex.feature_lists.feature_list["tokens"]
    fl_labels = ex.feature_lists.feature_list["labels"]
    label = labels[-1]
    for token in sequence:
        fl_tokens.feature.add().int64_list.value.append(token)
        fl_labels.feature.add().int64_list.value.append(label)
    return ex
filepath='processed_uniprot.tfrecord'
# Write all examples into a TFRecords file
writer = tf.python_io.TFRecordWriter(filepath)

#     for sequence, label_sequence in zip(sequences, label_sequences):
for i,rec in seq_records.iterrows():
    ex = make_example(rec.seq, [rec.fam])
    writer.write(ex.SerializeToString())

writer.close()
print("Wrote to {}".format(filepath))


Wrote to processed_uniprot.tfrecord


In [20]:
tf.reset_default_graph()
# A single serialized example
# (You can read this from a file using TFRecordReader)
filequeue = tf.train.string_input_producer([filepath],num_epochs=num_epochs)
reader = tf.TFRecordReader()
_,serial_ex=reader.read(filequeue)

# Parse the example (returns a dictionary of tensors)
context_parsed, sequence_parsed = tf.parse_single_sequence_example(
    serialized=serial_ex,
    context_features=context_features,
    sequence_features=sequence_features
)
# context_parsed is now a Tensor !
context = tf.contrib.learn.run_n(context_parsed, n=4, feed_dict=None)

#     output_dict: A dict mapping string names to tensors to run. Must all be from the same graph.
#     feed_dict: dict of input values to feed each run.
#     restore_checkpoint_path: A string containing the path to a checkpoint to restore.
#     n: Number of times to repeat.

print(context)
sequence = tf.contrib.learn.run_n(sequence_parsed, n=2, feed_dict=None)
print(sequence)

[{'length': 368}, {'length': 337}, {'length': 338}, {'length': 338}]
[{'tokens': array([17, 13,  7, 12,  1,  5, 14, 17, 12, 20, 19,  4, 12, 22,  5,  4, 23,
        9, 12, 14, 14, 16,  5,  1, 19,  9,  3, 20, 16,  5,  7,  9, 14,  4,
        6, 18,  1,  9,  1, 14,  6, 17,  4, 25,  8,  7, 12,  1,  5,  6, 18,
       14,  1, 22,  1, 11,  6, 13,  1, 18, 20, 18,  7, 14, 18,  9, 20,  6,
        4, 16,  4, 18,  9, 22, 13, 19,  7,  7,  1, 20,  7,  1,  8,  5, 22,
       20,  1,  6,  3, 12,  1,  4, 16,  7,  5,  1,  6, 12, 22, 16,  9, 16,
       25, 25, 16,  7,  6,  4, 18,  4, 12, 18, 23, 18, 20,  7, 22, 11, 12,
       22, 16, 22, 13,  3,  4, 19, 19, 14, 14,  6, 22, 12, 20, 11,  5,  1,
       12,  5,  4,  1, 25,  5, 11,  1, 18,  5,  4, 14,  9, 18, 22, 11,  7,
       12, 12,  9, 20, 14, 16, 19, 14, 16, 12,  7, 20,  9, 13,  4, 18, 11,
       20, 12, 18, 20, 22, 22, 19,  6,  9, 14,  5, 11, 18,  9,  8, 12, 22,
        3,  4,  5,  9, 25,  1,  1, 20, 22,  6, 19, 17, 16,  7,  6,  9, 19,
        9,  1,  5, 

In [44]:
a={'someKey':tf.Variable(5)}
b=tf.Variable([5])
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
# print(tf.contrib.learn.run_n(a,n=4,feed_dict=None))
# tf.Session().run([init,a])
a = tf.Print(b[-1],[b])
a.eval()

5

'1.2.0'